# 3D volumes definitions with UWGeodynamics

Claire Mallard (claire.mallard@sydney.edu.au)

Romain Beucher (romain.beucher@anu.edu.au)

In [ ]:
from underworld import UWGeodynamics as GEO
from UWGeodynamics import visualisation as vis

Load the unit registry to facilitate definition of quantities.

In [ ]:
u = GEO.UnitRegistry

In [ ]:
Model = GEO.Model(elementRes=(34, 34, 12), 
                  minCoord=(0. * u.kilometer, -0. * u.kilometer, -2880. * u.kilometer), 
                  maxCoord=(9000. * u.kilometer, 2000. * u.kilometer, 20. * u.kilometer), 
                  gravity=(0.0 ,0.0, -9.81 * u.meter / u.second**2))

# Create a 3D volume

HalfSpaces can be used to divide the space in two domains. The divide is a plan that can be defined using the by its normal vector. The convention is to keep the domain opposite to the direction the normal is pointing towards.

In the following example we create 4 halfSpaces and combine them to define a 3D volume: 

![title](./images/3D_hafspaces.png)

In [ ]:
halfspace1 = GEO.shapes.HalfSpace(normal=(-1.,0.,1.),origin=(4000. * u.kilometer, 0. * u.kilometer, -1000. * u.kilometer))
halfspace2 = GEO.shapes.HalfSpace(normal=(0.,0.,1.),origin=(7000. * u.kilometer, 1000. * u.kilometer, 0. * u.kilometer))
halfspace3 = GEO.shapes.HalfSpace(normal=(1.,0.,0.),origin=(9000. * u.kilometer, 1000. * u.kilometer, -500. * u.kilometer))
halfspace4 = GEO.shapes.HalfSpace(normal=(0.,0.,-1.),origin=(6500. * u.kilometer, 1000. * u.kilometer, -1000. * u.kilometer))

By combining the HalfSpace we can define a 3D volume that corresponds to the array where the domains overlap.

In [ ]:
CompositeShape = halfspace1 & halfspace2 & halfspace3 & halfspace4
volume3D = Model.add_material(name="Polygon", shape=CompositeShape)

In [ ]:
Fig = vis.Figure(resolution=(1200,600), axis=True)
Fig.Points(Model.swarm, Model.materialField, cullface=False, opacity=1.)
Fig.Mesh(Model.mesh)

# rotate view from default
lv = Fig.window()
lv.rotate('x', -70)
lv.rotate('y', -10)
lv.redisplay()

## Combining Layer and halfSpaces

A quickest approach to the above example is to define first a layer and then limit the extent of that layer using a halfspace:

![title](./images/3D_layer.png)

In [ ]:
AirShape = GEO.shapes.Layer3D(top=Model.top, bottom=0. * u.kilometer)
MaterialShape = GEO.shapes.Layer3D(top=0.0, bottom=-1000. * u.kilometer)
halfspace1 = GEO.shapes.HalfSpace(normal=(-1.,0.,1.),origin=(4000. * u.kilometer, 0. * u.kilometer, -1000. * u.kilometer))
PolygonShape = MaterialShape & halfspace1

In [ ]:
Air = Model.add_material(name="Air", shape=AirShape)
Layer = Model.add_material(name="Layer", shape=MaterialShape)
Polygon = Model.add_material(name="Polygon", shape=CompositeShape)

In [ ]:
Fig = vis.Figure(resolution=(1200,600), axis=True)
Fig.Points(Model.swarm, Model.materialField, cullface=False, opacity=1.)
Fig.Mesh(Model.mesh)

lv = Fig.window()
lv.rotate('x', -70)
lv.rotate('y', -10)
lv.redisplay()

## Definition of a 3D volume from multiples 3D volumes

For more complex cases, it may be useful to subdivide the volume of interest into smaller volumes easier to define.

In the following example we define two volumes and then combine them so that the volume defines the extent of one single material.

### Volume 1
The first volume is identical to the volume we have defined previously

In [ ]:
top    = GEO.shapes.HalfSpace(normal=(0.,0.,1.), origin=(2000.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))
right  = GEO.shapes.HalfSpace(normal=(1.,0.,0.), origin=(9000.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))
bottom = GEO.shapes.HalfSpace(normal=(0.,0.,-1.), origin=(2000.*u.kilometer,1000.*u.kilometer,-1000.*u.kilometer))
left   = GEO.shapes.HalfSpace(normal=(-1.,0.,1.), origin=(4500.*u.kilometer,1000.*u.kilometer,-500.*u.kilometer))
CompositeShape_Polygon1 = top & right & bottom & left  

Lets create a dummy material for visualisation purpose only:

In [ ]:
dummy = Model.add_material(name="dummy", shape=CompositeShape_Polygon1, reset=True)

In [ ]:
Fig = vis.Figure(resolution=(1200,600), axis=True)
Fig.Points(Model.swarm, Model.materialField, cullface=False, opacity=1.)
Fig.Mesh(Model.mesh)
lv = Fig.window()
lv.rotate('x', -70)
lv.rotate('y', -10)
lv.redisplay()

### Volume 2

Now lets do the second volume 

In [ ]:
top2    = GEO.shapes.HalfSpace(normal=(0.,0.,1.), origin=(5000.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))
right2  = GEO.shapes.HalfSpace(normal=(1.,0.,-1.), origin=(4500.*u.kilometer,1000.*u.kilometer,-500.*u.kilometer))
bottom2 = GEO.shapes.HalfSpace(normal=(-1.,0.,0.), origin=(3000.*u.kilometer,1000.*u.kilometer,-2000.*u.kilometer))
left2   = GEO.shapes.HalfSpace(normal=(-1.,0.,1.), origin=(3422.*u.kilometer,1180.*u.kilometer,-320.*u.kilometer))
CompositeShape_Polygon2 = top2 & right2 & bottom2 & left2

Again, Lets create a dummy material for visualisation purpose only:

In [ ]:
dummy = Model.add_material(name="dummy", shape=CompositeShape_Polygon2, reset=True)

In [ ]:
Fig = vis.Figure(resolution=(1200,600), axis=True)
Fig.Points(Model.swarm, Model.materialField, cullface=False, opacity=1.)
Fig.Mesh(Model.mesh)
lv = Fig.window()
lv.rotate('x', -70)
lv.rotate('y', -10)
lv.redisplay()

### Combine Volume 1 and Volume2

In [ ]:
CompositeShape = CompositeShape_Polygon1 | CompositeShape_Polygon2

In [ ]:
Material = Model.add_material(name="Material", shape=CompositeShape)

In [ ]:
Fig = vis.Figure(resolution=(1200,600), axis=True)
Fig.Points(Model.swarm, Model.materialField, cullface=False, opacity=1.)
Fig.Mesh(Model.mesh)
lv = Fig.window()
lv.rotate('x', -70)
lv.rotate('y', -10)
lv.redisplay()